#매칭시킨 학습데이터 가져오기

In [2]:
import pandas as pd
df=pd.read_csv('df.csv')

In [3]:
df.head(10)

,Unnamed: 0,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
0,0,0013cc385424,"Hi, i'm Isaac, i'm going to be writing about h...","Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate
1,1,9704a709b505,"Hi, i'm Isaac, i'm going to be writing about h...","On my perspective, I think that the face is a ...",Position,Adequate
2,2,c22adee811b6,"Hi, i'm Isaac, i'm going to be writing about h...",I think that the face is a natural landform be...,Claim,Adequate
3,3,a10d361e54e4,"Hi, i'm Isaac, i'm going to be writing about h...","If life was on Mars, we would know by now. The...",Evidence,Adequate
4,4,db3e453ec4e2,"Hi, i'm Isaac, i'm going to be writing about h...",People thought that the face was formed by ali...,Counterclaim,Adequate
5,5,36a565e45db7,"Hi, i'm Isaac, i'm going to be writing about h...","though some say that life on Mars does exist, ...",Rebuttal,Ineffective
6,6,fb65fe816ba3,"Hi, i'm Isaac, i'm going to be writing about h...","It says in paragraph 7, on April 5, 1998, Mars...",Evidence,Adequate
7,7,4e472e2584fa,"Hi, i'm Isaac, i'm going to be writing about h...",Everyone who thought it was made by alieans ev...,Counterclaim,Adequate
8,8,28a94d3ee425,"Hi, i'm Isaac, i'm going to be writing about h...",Though people were not satified about how the ...,Concluding Statement,Adequate
9,9,d226f06362f5,Limiting the usage of cars has personal and pr...,Limiting the usage of cars has personal and pr...,Lead,Effective


In [3]:
df['discourse_type'].unique()

array(['Lead', 'Position', 'Claim', 'Evidence', 'Counterclaim',
       'Concluding Statement', 'Rebuttal'], dtype=object)

#적절하지 못한 논증은 제거

In [4]:

print("Ineffective 컬럼 제거 전 ",df['discourse_effectiveness'].unique())

df = df.drop(df[df['discourse_effectiveness'] == 'Ineffective'].index)

print("Ineffective 컬럼 제거 후 ",df['discourse_effectiveness'].unique())


Ineffective 컬럼 제거 전  ['Adequate' 'Ineffective' 'Effective']
Ineffective 컬럼 제거 후  ['Adequate' 'Effective']


#적절하지 못한 논증을 제거

In [3]:
print("필요없는 데이터 제거후 갯수",df.shape[0])

필요없는 데이터 제거후 갯수 30303


#wordcount세기

In [5]:
count=0
## 워드 카운트 컬럼 생성
df['wordcount']=0
##워드 카운트 계산하기
df['wordcount'] = df['discourse_text'].apply(lambda x: len(x.split()))

#문서 요약

In [61]:
from transformers import pipeline
summarizer1 = pipeline('summarization',max_length=70, min_length=20, do_sample=False)

No model was supplied, defaulted to t5-small and revision d769bba (https://huggingface.co/t5-small).
Using a pipeline without specifying a model name and revision in production is not recommended.


All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


#wordcount이상치

In [ ]:
df['wordcount'].describe()

count    30303.000000
mean        43.783388
std         43.509495
min          1.000000
25%         15.000000
50%         27.000000
75%         57.000000
max        542.000000
Name: wordcount, dtype: float64

In [6]:
wordcount_iqr = df['wordcount'].quantile(0.75) - df['wordcount'].quantile(0.25)
wordcount_iqr=wordcount_iqr*1.5+df['wordcount'].quantile(0.75)
print("IQR of 'wordcount' column:", wordcount_iqr)


IQR of 'wordcount' column: 120.0


In [7]:
df_on = df[df['wordcount'] >= 120]

# Create df_under where wordcount is less than 120
df_under = df[df['wordcount'] < 120]

##2043개의 이상치데이터 문서 요약

In [8]:

from tqdm import tqdm
sample = list(df_on['discourse_text'])

for i in tqdm(range(len(sample))):
  sample[i] = summarizer1(sample[i])[0]['summary_text']


100%|████████████████████████████████████████████████████████████████████████████| 2043/2043 [6:21:59<00:00, 11.22s/it]


In [9]:
df_on['discourse_text']=sample

C:\Users\OWNER\AppData\Local\Temp\ipykernel_24792\3496437381.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_on['discourse_text']=sample


In [15]:
concatenated_df = pd.concat([df_on, df_under])
df = concatenated_df.sort_index()
df.head()

,Unnamed: 0,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,wordcount
0,0,0013cc385424,"Hi, i'm Isaac, i'm going to be writing about h...","Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,67
1,1,9704a709b505,"Hi, i'm Isaac, i'm going to be writing about h...","On my perspective, I think that the face is a ...",Position,Adequate,41
2,2,c22adee811b6,"Hi, i'm Isaac, i'm going to be writing about h...",I think that the face is a natural landform be...,Claim,Adequate,21
3,3,a10d361e54e4,"Hi, i'm Isaac, i'm going to be writing about h...","If life was on Mars, we would know by now. The...",Evidence,Adequate,72
4,4,db3e453ec4e2,"Hi, i'm Isaac, i'm going to be writing about h...",People thought that the face was formed by ali...,Counterclaim,Adequate,18
6,6,fb65fe816ba3,"Hi, i'm Isaac, i'm going to be writing about h...","It says in paragraph 7, on April 5, 1998, Mars...",Evidence,Adequate,37
7,7,4e472e2584fa,"Hi, i'm Isaac, i'm going to be writing about h...",Everyone who thought it was made by alieans ev...,Counterclaim,Adequate,34
8,8,28a94d3ee425,"Hi, i'm Isaac, i'm going to be writing about h...",Though people were not satified about how the ...,Concluding Statement,Adequate,43
9,9,d226f06362f5,Limiting the usage of cars has personal and pr...,Limiting the usage of cars has personal and pr...,Lead,Effective,119
10,10,de347c859ab6,Limiting the usage of cars has personal and pr...,With so many things in this world that few peo...,Position,Effective,41


#감성분석

In [17]:
from transformers import pipeline
#
pipe = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment-latest")


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
from tqdm import tqdm
sample2 = list(concatenated_df['discourse_text'])

for i in tqdm(range(len(sample2))):
  
  sample2[i] = pipe(sample2[i])
  sample2[i] = sample2[i][0]['label']


100%|██████████████████████████████████████████████████████████████████████████| 30303/30303 [2:10:07<00:00,  3.88it/s]


In [20]:
df['discourse_Emotion']=sample2

### claim_label분석

In [6]:
# Use a pipeline as a high-level helper
from transformers import pipeline

Claim_pipe = pipeline("text-classification", model="ThinkCERCA/claim_hugging")

Some layers from the model checkpoint at ThinkCERCA/claim_hugging were not used when initializing TFBertForSequenceClassification: ['dropout_265']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at ThinkCERCA/claim_hugging.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [ ]:
from tqdm import tqdm
sample = list(df_wordcount['discourse_text'])

for i in tqdm(range(len(sample))):
  sample[i] = Claim_pipe(sample[i])[0]['label']

In [ ]:
df['claim_label']=sample

### 상위코드는 시간이 오래걸리므로 결과를 저장한 csv가져오기

In [ ]:
import pandas as pd
df=pd.read_csv('df_claim_label.csv')

# 상위 단어 추출

In [3]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\OWNER\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\OWNER\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\OWNER\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\OWNER\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\OWNER\AppData\Roaming\nltk_data...
[nltk_data]    | 

[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package mte_teip5 to
[nltk_data]    |     C:\Users\OWNER\AppData\Roaming\nltk_data...
[nltk_data]    |   Package mte_teip5 is already up-to-date!
[nltk_data]    | Downloading package mwa_ppdb to
[nltk_data]    |     C:\Users\OWNER\AppData\Roaming\nltk_data...
[nltk_data]    |   Package mwa_ppdb is already up-to-date!
[nltk_data]    | Downloading package names to
[nltk_data]    |     C:\Users\OWNER\AppData\Roaming\nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Downloading package nombank.1.0 to
[nltk_data]    |     C:\Users\OWNER\AppData\Roaming\nltk_data...
[nltk_data]    |   Package nombank.1.0 is already up-to-date!
[nltk_data]    | Downloading package nonbreaking_prefixes to
[nltk_data]    |     C:\Users\OWNER\AppData\Roaming\nltk_data...
[nltk_data]    |   Package nonbreaking_prefixes is already up-to-date!
[nltk_data]    | Downloading package nps_chat

[nltk_data]    |   Package universal_treebanks_v20 is already up-to-
[nltk_data]    |       date!
[nltk_data]    | Downloading package vader_lexicon to
[nltk_data]    |     C:\Users\OWNER\AppData\Roaming\nltk_data...
[nltk_data]    |   Package vader_lexicon is already up-to-date!
[nltk_data]    | Downloading package verbnet to
[nltk_data]    |     C:\Users\OWNER\AppData\Roaming\nltk_data...
[nltk_data]    |   Package verbnet is already up-to-date!
[nltk_data]    | Downloading package verbnet3 to
[nltk_data]    |     C:\Users\OWNER\AppData\Roaming\nltk_data...
[nltk_data]    |   Package verbnet3 is already up-to-date!
[nltk_data]    | Downloading package webtext to
[nltk_data]    |     C:\Users\OWNER\AppData\Roaming\nltk_data...
[nltk_data]    |   Package webtext is already up-to-date!
[nltk_data]    | Downloading package wmt15_eval to
[nltk_data]    |     C:\Users\OWNER\AppData\Roaming\nltk_data...
[nltk_data]    |   Package wmt15_eval is already up-to-date!
[nltk_data]    | Downloadin

True

In [4]:
lead_df=df[df.discourse_type=='Lead']
Position_df=df[df.discourse_type=='Position']
Claim_df=df[df.discourse_type=='Claim']
Evidence_df=df[df.discourse_type=='Evidence']
Counterclaim_df=df[df.discourse_type=='Counterclaim']
Concluding_Statement_df=df[df.discourse_type=='Concluding Statement']
Rebuttal_df=df[df.discourse_type=='Rebuttal']

### 각상위 단어 600개씩 추출하기

In [5]:
import nltk
from nltk.tokenize import word_tokenize

token_lst=[]
for text in lead_df.discourse_text:

  tokens = word_tokenize(text)
  token_lst.append(tokens)
from collections import Counter
from itertools import chain

# 리스트 안의 리스트를 평탄화
flattened_lst = list(chain.from_iterable(token_lst))

# 각 요소의 개수 세기
token_counts = Counter(flattened_lst)

# 개수에 따라 내림차순으로 정렬된 (값, 개수) 튜플 리스트
sorted_token_counts = sorted(token_counts.items(), key=lambda x: x[1], reverse=True)

# 정렬된 결과 출력
lst=[]
cnt=0
for value, count in sorted_token_counts:
    if cnt==600:
      break
    cnt=cnt+1
    lst.append(value)

In [6]:

token_lst=[]
for text in Position_df.discourse_text:

  tokens = word_tokenize(text)
  token_lst.append(tokens)


# 리스트 안의 리스트를 평탄화
flattened_lst = list(chain.from_iterable(token_lst))

# 각 요소의 개수 세기
token_counts = Counter(flattened_lst)

# 개수에 따라 내림차순으로 정렬된 (값, 개수) 튜플 리스트
sorted_token_counts = sorted(token_counts.items(), key=lambda x: x[1], reverse=True)

# 정렬된 결과 출력

cnt=0
for value, count in sorted_token_counts:
    if cnt==600:
      break
    cnt=cnt+1
    lst.append(value)

In [7]:
import nltk
from nltk.tokenize import word_tokenize

token_lst=[]
for text in Claim_df.discourse_text:

  tokens = word_tokenize(text)
  token_lst.append(tokens)
from collections import Counter
from itertools import chain

# 리스트 안의 리스트를 평탄화
flattened_lst = list(chain.from_iterable(token_lst))

# 각 요소의 개수 세기
token_counts = Counter(flattened_lst)

# 개수에 따라 내림차순으로 정렬된 (값, 개수) 튜플 리스트
sorted_token_counts = sorted(token_counts.items(), key=lambda x: x[1], reverse=True)

# 정렬된 결과 출력
cnt=0
for value, count in sorted_token_counts:
    if cnt==600:
      break
    cnt=cnt+1
    lst.append(value)

In [8]:
import nltk
from nltk.tokenize import word_tokenize

token_lst=[]
for text in Evidence_df.discourse_text:

  tokens = word_tokenize(text)
  token_lst.append(tokens)
from collections import Counter
from itertools import chain

# 리스트 안의 리스트를 평탄화
flattened_lst = list(chain.from_iterable(token_lst))

# 각 요소의 개수 세기
token_counts = Counter(flattened_lst)

# 개수에 따라 내림차순으로 정렬된 (값, 개수) 튜플 리스트
sorted_token_counts = sorted(token_counts.items(), key=lambda x: x[1], reverse=True)

# 정렬된 결과 출력
cnt=0
for value, count in sorted_token_counts:
    if cnt==600:
      break
    cnt=cnt+1
    lst.append(value)

In [9]:
import nltk
from nltk.tokenize import word_tokenize

token_lst=[]
for text in Counterclaim_df.discourse_text:

  tokens = word_tokenize(text)
  token_lst.append(tokens)
from collections import Counter
from itertools import chain

# 리스트 안의 리스트를 평탄화
flattened_lst = list(chain.from_iterable(token_lst))

# 각 요소의 개수 세기
token_counts = Counter(flattened_lst)

# 개수에 따라 내림차순으로 정렬된 (값, 개수) 튜플 리스트
sorted_token_counts = sorted(token_counts.items(), key=lambda x: x[1], reverse=True)

# 정렬된 결과 출력
cnt=0
for value, count in sorted_token_counts:
    if cnt==600:
      break
    cnt=cnt+1
    lst.append(value)

In [10]:
import nltk
from nltk.tokenize import word_tokenize

token_lst=[]
for text in Concluding_Statement_df.discourse_text:

  tokens = word_tokenize(text)
  token_lst.append(tokens)
from collections import Counter
from itertools import chain

# 리스트 안의 리스트를 평탄화
flattened_lst = list(chain.from_iterable(token_lst))

# 각 요소의 개수 세기
token_counts = Counter(flattened_lst)

# 개수에 따라 내림차순으로 정렬된 (값, 개수) 튜플 리스트
sorted_token_counts = sorted(token_counts.items(), key=lambda x: x[1], reverse=True)

# 정렬된 결과 출력
cnt=0
for value, count in sorted_token_counts:
    if cnt==600:
      break
    cnt=cnt+1
    lst.append(value)

In [11]:
import nltk
from nltk.tokenize import word_tokenize

token_lst=[]
for text in Rebuttal_df.discourse_text:

  tokens = word_tokenize(text)
  token_lst.append(tokens)
from collections import Counter
from itertools import chain

# 리스트 안의 리스트를 평탄화
flattened_lst = list(chain.from_iterable(token_lst))

# 각 요소의 개수 세기
token_counts = Counter(flattened_lst)

# 개수에 따라 내림차순으로 정렬된 (값, 개수) 튜플 리스트
sorted_token_counts = sorted(token_counts.items(), key=lambda x: x[1], reverse=True)

# 정렬된 결과 출력

cnt=0
for value, count in sorted_token_counts:
    if cnt==600:
      break
    cnt=cnt+1
    lst.append(value)

In [12]:
lst=list(set(lst))

In [13]:
lst

['national',
 'happy',
 'comfortable',
 'kind',
 'guidelines',
 'known',
 'allowed',
 'saying',
 'facts',
 'longer',
 'everyday',
 'increase',
 'alien',
 'child',
 'seems',
 'grow',
 'America',
 'two',
 'than',
 'given',
 'Everyone',
 'against',
 'valuable',
 'extremely',
 'Finally',
 'courses',
 'them',
 'essay',
 'debate',
 'project',
 'budget',
 'direct',
 'for',
 'now',
 'easy',
 'world',
 'real',
 'sure',
 'attending',
 'outcome',
 'nothing',
 'Those',
 'dangerous',
 'balance',
 'likely',
 'It',
 'conferencing',
 'play',
 'computers',
 'yourself',
 'wont',
 'times',
 'My',
 'means',
 '1976',
 'thing',
 'rid',
 'Source',
 'safety',
 'eyes',
 'busy',
 'Most',
 'scientists',
 'daily',
 'the',
 'are',
 'principal',
 'material',
 'prevent',
 '2001',
 'high',
 'setting',
 'every',
 'slate',
 'irrational',
 'these',
 'last',
 'driver',
 'President',
 'got',
 'pollution',
 'Would',
 '...',
 'assure',
 'ideas',
 'However',
 'worth',
 'why',
 'those',
 'education',
 'helping',
 'choices',
 

In [14]:
for i in lst:
  df[i]=0

C:\Users\OWNER\AppData\Local\Temp\ipykernel_2448\4237062562.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[i]=0
C:\Users\OWNER\AppData\Local\Temp\ipykernel_2448\4237062562.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[i]=0
C:\Users\OWNER\AppData\Local\Temp\ipykernel_2448\4237062562.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de

C:\Users\OWNER\AppData\Local\Temp\ipykernel_2448\4237062562.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[i]=0
C:\Users\OWNER\AppData\Local\Temp\ipykernel_2448\4237062562.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[i]=0
C:\Users\OWNER\AppData\Local\Temp\ipykernel_2448\4237062562.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de

In [15]:
for word in lst:
    df[word] = df['discourse_text'].apply(lambda text: text.count(word))

#학습 시작

In [16]:
lst=['Unnamed: 0','discourse_id','essay_id','discourse_text']
for i in lst:
  df = df.drop(columns=i)

In [17]:
df.head()

,Unnamed: 0.1,discourse_type,discourse_effectiveness,wordcount,discourse_Emotion,claim_label,national,happy,comfortable,kind,...,Even,bored,just,experiences,lot,coding,2,planets,avoid,selecting
0,0,Lead,Adequate,67,negative,Claim,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,1,Position,Adequate,41,positive,Claim,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,Claim,Adequate,21,positive,Claim,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,Evidence,Adequate,72,negative,Claim,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,Counterclaim,Adequate,18,negative,Not Claim,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
df.iloc[:,5:]

,claim_label,national,happy,comfortable,kind,guidelines,known,allowed,saying,facts,...,Even,bored,just,experiences,lot,coding,2,planets,avoid,selecting
0,Claim,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,Claim,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Claim,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Claim,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
4,Not Claim,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30298,Not Claim,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30299,Not Claim,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30300,Not Claim,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30301,Not Claim,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
mapping = {value: index for index, value in enumerate(df.discourse_type.unique())}

# 변환
df['discourse_type'] = df['discourse_type'].replace(mapping)

In [20]:
one_hot_encoded = pd.get_dummies(df['discourse_effectiveness'], prefix='discourse')
df=pd.concat([df,one_hot_encoded], axis=1)
df = df.drop(columns=['discourse_effectiveness'])

In [21]:
one_hot_encoded = pd.get_dummies(df['discourse_Emotion'], prefix='discourse')
df=pd.concat([df,one_hot_encoded], axis=1)
df = df.drop(columns=['discourse_Emotion'])

In [22]:
one_hot_encoded = pd.get_dummies(df['claim_label'], prefix='discourse')
df=pd.concat([df,one_hot_encoded], axis=1)
df = df.drop(columns=['claim_label'])

In [23]:
df

,Unnamed: 0.1,discourse_type,wordcount,national,happy,comfortable,kind,guidelines,known,allowed,...,planets,avoid,selecting,discourse_Adequate,discourse_Effective,discourse_negative,discourse_neutral,discourse_positive,discourse_Claim,discourse_Not Claim
0,0,0,67,0,0,0,0,0,0,0,...,0,0,0,True,False,True,False,False,True,False
1,1,1,41,0,0,0,0,0,0,0,...,0,0,0,True,False,False,False,True,True,False
2,2,2,21,0,0,0,0,0,0,0,...,0,0,0,True,False,False,False,True,True,False
3,3,3,72,0,0,0,0,0,0,0,...,0,0,0,True,False,True,False,False,True,False
4,4,4,18,0,0,0,0,0,0,0,...,0,0,0,True,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30298,30298,3,10,0,0,0,0,0,0,0,...,0,0,0,True,False,False,False,True,False,True
30299,30299,4,67,0,0,0,0,0,0,0,...,0,0,0,True,False,True,False,False,False,True
30300,30300,2,21,0,0,0,0,0,0,0,...,0,0,0,True,False,False,False,True,False,True
30301,30301,2,9,0,0,0,0,0,0,0,...,0,0,0,True,False,True,False,False,False,True


In [24]:
df.head()

,Unnamed: 0.1,discourse_type,wordcount,national,happy,comfortable,kind,guidelines,known,allowed,...,planets,avoid,selecting,discourse_Adequate,discourse_Effective,discourse_negative,discourse_neutral,discourse_positive,discourse_Claim,discourse_Not Claim
0,0,0,67,0,0,0,0,0,0,0,...,0,0,0,True,False,True,False,False,True,False
1,1,1,41,0,0,0,0,0,0,0,...,0,0,0,True,False,False,False,True,True,False
2,2,2,21,0,0,0,0,0,0,0,...,0,0,0,True,False,False,False,True,True,False
3,3,3,72,0,0,0,0,0,0,0,...,0,0,0,True,False,True,False,False,True,False
4,4,4,18,0,0,0,0,0,0,0,...,0,0,0,True,False,True,False,False,False,True


In [25]:
columns_list = df.columns[1:].tolist()

In [26]:
df.isnull().sum()

Unnamed: 0.1           0
discourse_type         0
wordcount              0
national               0
happy                  0
                      ..
discourse_negative     0
discourse_neutral      0
discourse_positive     0
discourse_Claim        0
discourse_Not Claim    0
Length: 1007, dtype: int64

In [27]:
##라이브러리 임포트
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

df['discourse_type'] = df['discourse_type'].astype(int)

X = df[columns_list]

y = df[['discourse_type']].values.ravel()

## 학습,테스트,트레인,테스트출력
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

##학습할 컬럼 지정
X_train_cat = X_train.loc[:, columns_list]
X_test_cat = X_test.loc[:, columns_list]

In [28]:
!pip install xgboost

In [29]:
import xgboost as xgb
xgb_classifier = xgb.XGBClassifier(seed=41)
models ={"XGBClassifier Classifier": xgb.XGBClassifier(seed=41) }

In [33]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
results = []
for name, model in models.items():
	kf = KFold(n_splits=5, random_state=41, shuffle=True)
	cv_results = cross_val_score(model, X_train_cat, y_train, cv=kf)
	results.append(cv_results)
 #500개는  0.6879797695690377
 #Column: argue, Average Value: 0.005922440439462662
 #Column: ?, Average Value: 0.007152101956307888
 # Column: Seeking, Average Value: 0.007834510877728462
 # Column: should, Average Value: 0.009158594533801079
 # Column: wordcount, Average Value: 0.012555768713355064
 # Column: conclusion, Average Value: 0.023079317063093185

 #600개는 0.6888872837582789
#  Column: Some, Average Value: 0.004021185450255871
#  Column: despite, Average Value: 0.004098579287528992
#  Column: benefit, Average Value: 0.004205523990094662
#  Column: ., Average Value: 0.004374821204692125
#  Column: example, Average Value: 0.004457774106413126
#  Column: argue, Average Value: 0.005450223572552204
#  Column: ?, Average Value: 0.006851424928754568
#  Column: should, Average Value: 0.007944924756884575
#  Column: wordcount, Average Value: 0.011371307075023651
#Column: conclusion, Average Value: 0.020326068624854088
	print("{} Mean Cross-Validation Accuracy Score: {}".format(name, cv_results.mean()))

XGBClassifier Classifier Mean Cross-Validation Accuracy Score: 0.6910737438973842


In [30]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, random_state=41, shuffle=True)
y_train_df = pd.DataFrame(y_train, columns=['discourse_type'])
for train_index, test_index in kf.split(X_train_cat):
    X_train, X_test = X_train_cat.iloc[train_index,:], X_train_cat.iloc[test_index,:]
    y_train, y_test = y_train_df.iloc[train_index,:], y_train_df.iloc[test_index,:]

In [31]:
import pandas as pd
from sklearn.model_selection import KFold
from xgboost import XGBClassifier

# KFold 객체 생성
kf = KFold(n_splits=5, random_state=41, shuffle=True)

# 결과를 저장할 DataFrame 생성
result_df = pd.DataFrame(columns=X_train_cat.columns)

# 각 fold에 대해 반복
for train_index, test_index in kf.split(X_train_cat):
    X_train, X_test = X_train_cat.iloc[train_index], X_train_cat.iloc[test_index]
    y_train, y_test = y_train_df.iloc[train_index], y_train_df.iloc[test_index]

    # XGBClassifier 모델 생성 및 학습
    model = XGBClassifier()
    model.fit(X_train, y_train)

    # 기능 중요도를 DataFrame에 추가
    result_df = pd.concat([result_df, pd.DataFrame([model.feature_importances_], columns=X_train_cat.columns)], ignore_index=True)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# 시각화 설정
plt.figure(figsize=(600, 120))
sns.set(style="whitegrid")

# 막대 그래프 생성
sns.barplot(data=result_df, orient='v')

# 그래프 제목과 라벨 설정
plt.title('Feature Importance')
plt.xlabel('Importance')
plt.ylabel('Features')

# 그래프 표시

In [32]:
avg_values = result_df.mean()

# Sort the columns based on the average values
sorted_columns = avg_values.sort_values().index.tolist()

In [33]:
avg_values = result_df.mean()

# Sort the columns based on the average values and retrieve both column names and their corresponding average values
sorted_columns_and_values = [(col, avg_values[col]) for col in avg_values.sort_values().index]

# Print the sorted column names and their corresponding average values
print("Columns in order of average values:")
for col, avg_value in sorted_columns_and_values:
    print(f"Column: {col}, Average Value: {avg_value}")

Columns in order of average values:
Column: assignment, Average Value: 0.0
Column: were, Average Value: 0.0
Column: elect, Average Value: 0.0
Column: lost, Average Value: 0.0
Column: FACS, Average Value: 0.0
Column: In, Average Value: 0.0
Column: mesa, Average Value: 0.0
Column: any, Average Value: 0.0
Column: across, Average Value: 0.0
Column: prove, Average Value: 0.0
Column: Plumer, Average Value: 0.0
Column: want, Average Value: 0.0
Column: sounds, Average Value: 0.0
Column: even, Average Value: 0.0
Column: stated, Average Value: 0.0
Column: makes, Average Value: 0.0
Column: car, Average Value: 0.0
Column: less, Average Value: 0.0
Column: formation, Average Value: 0.0
Column: Not, Average Value: 0.0
Column: seek, Average Value: 0.0
Column: looks, Average Value: 0.0
Column: Though, Average Value: 0.0
Column: friend, Average Value: 0.0
Column: Policy, Average Value: 0.0
Column: purpose, Average Value: 0.0
Column: show, Average Value: 0.0
Column: college, Average Value: 0.0
Column: ha

### 

In [34]:
sorted_columns.append("discourse_type")

In [35]:
df = df.reindex(columns=sorted_columns)

In [37]:
df=df.iloc[:,::-1]

In [40]:
df = df.loc[:, ~df.columns.duplicated()]
df

,discourse_type,wordcount,.,a,discourse_Claim,I,he,i,the,discourse_negative,...,prove,across,any,mesa,In,FACS,lost,elect,were,assignment
0,0,67,3,28,True,1,4,19,3,True,...,0,0,0,0,0,0,0,0,0,0
1,1,41,1,20,True,5,3,10,3,False,...,0,0,1,0,1,0,0,0,0,0
2,2,21,0,10,True,1,2,4,2,False,...,0,0,0,0,0,0,0,0,0,0
3,3,72,4,33,True,5,4,21,2,True,...,0,0,0,0,1,0,0,0,0,0
4,4,18,1,9,False,0,3,2,3,True,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30298,3,10,1,4,False,0,0,1,0,False,...,0,0,0,0,0,0,0,0,0,0
30299,4,67,2,18,False,0,10,14,9,True,...,0,0,0,0,0,0,0,0,0,0
30300,2,21,0,6,False,0,1,6,1,False,...,0,0,1,0,0,0,0,0,0,0
30301,2,9,1,3,False,0,0,5,0,True,...,0,0,0,0,0,0,0,0,0,0


In [49]:
columns_list = df.columns[1:500].tolist()

In [41]:
##라이브러리 임포트
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

df['discourse_type'] = df['discourse_type'].astype(int)

X = df[columns_list]

y = df[['discourse_type']].values.ravel()

## 학습,테스트,트레인,테스트출력
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

##학습할 컬럼 지정
X_train_cat = X_train.loc[:, columns_list]
X_test_cat = X_test.loc[:, columns_list]

In [42]:
import xgboost as xgb
xgb_classifier = xgb.XGBClassifier(seed=41)
models ={"XGBClassifier Classifier": xgb.XGBClassifier(seed=41) }

In [43]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
results = []
for name, model in models.items():
	kf = KFold(n_splits=5, random_state=41, shuffle=True)
	cv_results = cross_val_score(model, X_train_cat, y_train, cv=kf)
	results.append(cv_results)
 #500개는  0.6879797695690377
 #Column: argue, Average Value: 0.005922440439462662
 #Column: ?, Average Value: 0.007152101956307888
 # Column: Seeking, Average Value: 0.007834510877728462
 # Column: should, Average Value: 0.009158594533801079
 # Column: wordcount, Average Value: 0.012555768713355064
 # Column: conclusion, Average Value: 0.023079317063093185

 #600개는 0.6888872837582789
#  Column: Some, Average Value: 0.004021185450255871
#  Column: despite, Average Value: 0.004098579287528992
#  Column: benefit, Average Value: 0.004205523990094662
#  Column: ., Average Value: 0.004374821204692125
#  Column: example, Average Value: 0.004457774106413126
#  Column: argue, Average Value: 0.005450223572552204
#  Column: ?, Average Value: 0.006851424928754568
#  Column: should, Average Value: 0.007944924756884575
#  Column: wordcount, Average Value: 0.011371307075023651
#Column: conclusion, Average Value: 0.020326068624854088
	print("{} Mean Cross-Validation Accuracy Score: {}".format(name, cv_results.mean()))

XGBClassifier Classifier Mean Cross-Validation Accuracy Score: 1.0
